In [1]:
# data 3/2/2024.
import socket
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle

def print_model_weights(model, message="Model weights"):
    print(message)
    for i, layer in enumerate(model.layers):
        weights, biases = layer.get_weights()
        print(f"Layer {i} - weights: {weights.shape} biases: {biases.shape}")
        print(f"Weights: \n{weights}\nBiases:\n{biases}\n")

def build_model():
    tf.keras.backend.clear_session()  # Clear any previous session.
    # NN model creation with input parameters
    model = tf.keras.Sequential([
        # First hidden layer with ReLU activation
        # weather 5 + roadcond 3 + dayofweek 7 + month 12 = e.g. 26 features
        tf.keras.layers.Dense(64, activation='relu', input_shape=(13,)),
        # Second hidden layer
        tf.keras.layers.Dense(32, activation='relu'),
        # Output layer with softmax activation
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    # Compile the model with optimizer, loss function, and metrics to evaluate
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    # print("model weights")
    # print (model.get_weights())
    weights = model.get_weights()
    # Calculate the total size of the weights
    total_elements = sum([np.prod(w.shape) for w in weights])
    print("Total number of elements in the GM weights:", total_elements)

    print("GM has been created and compiled successfully.")
    return model

def send_model(client_connection, model):
    # Serialize and send the model structure (architecture) to the client
    # print("Serializing the model structure to JSON format.")
    model_json = model.to_json()
    client_connection.sendall(model_json.encode('utf-8'))
    #print("Model structure sent to client. Awaiting acknowledgment...")
    # Wait for acknowledgment from the client before sending weights
    ack = client_connection.recv(1024).decode()
    if ack == "ACK_MODEL":
        # print("Acknowledgment received. Now sending initial model weights.")
        # Serialize and send the initial model weights
        initial_weights_data = pickle.dumps(model.get_weights())#initial_weights = pickle.dumps(model.get_weights())
        #print("***************************************************************")
        #print("initial_weights")
        #print(initial_weights)
        client_connection.sendall(len(initial_weights_data).to_bytes(8, byteorder='big'))
        client_connection.sendall(initial_weights_data)#client_connection.send(initial_weights)
        #print("Initial model weights have been sent to the client.")

def receive_and_update_model(client_connection, model):
    #print("Receiving updated model weights from the client.")
    data_size = int.from_bytes(client_connection.recv(8), byteorder='big')
    #Now receive the actual pickled data based on the size
    updated_weights_data = client_connection.recv(data_size)
    while len(updated_weights_data) < data_size:
        packet = client_connection.recv(data_size - len(updated_weights_data))
        if not packet: break
        updated_weights_data += packet

    updated_weights = pickle.loads(updated_weights_data)
    model.set_weights(updated_weights)
    print("Server model has been updated with the client's weights.")
    return updated_weights

#The function aims to aggregate (combine) the model weights received from multiple clients
# into a single set of weights. This aggregation is typically done by averaging the weights. 
# The process ensures that the global model learns from all the local datasets without actually having access to them.
def aggregate_weights(weight_list, trim_ratio=0.1):
    """Aggregates weights using a trimmed mean approach.
    Parameters:
    - weight_list: List of weight lists from each client model. Each element is a list of numpy arrays.
    - trim_ratio: Fraction of highest and lowest weights to trim before averaging.

    Returns:
    - Aggregated weights as a list of numpy arrays.  """
    # Check if weight_list is empty or contains None elements
    if not weight_list or None in weight_list:
        raise ValueError("Weight list is empty or contains None elements.")
    
    num_clients = len(weight_list)
    num_layers = len(weight_list[0])
    trim_count = int(trim_ratio * num_clients)
    
    aggregated_weights = []
    for layer_idx in range(num_layers):
        # Collect the weights for this layer from all clients
        layer_weights = np.array([client_weights[layer_idx] for client_weights in weight_list])
        # Sort the weights for each neuron/connection across all clients
        sorted_weights = np.sort(layer_weights, axis=0)
        # Trim the highest and lowest weights
        trimmed_weights = sorted_weights[trim_count:num_clients-trim_count]
        # Compute the mean of the trimmed weights
        mean_weights = trimmed_weights.mean(axis=0)
        aggregated_weights.append(mean_weights)
    
    print("Aggregated weights using trimmed mean.")
    return aggregated_weights
def val_weights(weights_list):
    for i, weights in enumerate(weights_list):
        for layer_weights in weights:
            if np.isnan(layer_weights).any():
                print(f"NaN found in weights from client {i}")
                return False  
    return True  # no nan values
# Server setup
host = '127.0.0.1' # change this back later '10.0.0.189'
port = 5300
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server_socket.bind((host, port))
server_socket.listen(2)
print(f"Server is listening on port {port}.")

client_counter = 0

# Build a model and send it to the client
print('Building GM')
model = build_model()
print("***************************************************************")
print("Clients please connect to server")

client_weights_list = []  # List to store weights from each client
client_conn = []  # List to store client connections
while client_counter < 3 :#change to devices 3
    conn, address = server_socket.accept()
    client_conn.append((conn, address))
    print("*************************************************")
    print(f"Established connection with client at {address}.")

    # send initial model to client
    send_model(conn, model)
    
    print("*******************")
    print(f"Initial model sent to client at {address}.")
    
    # Increment the client counter 
    client_counter += 1
    print(f"Ready for a new connection if any.")
    print("*******")


for conn, address in client_conn:    
    # Receive updated model weights from the client and update the server model
    print(f'Receiving client {address} local trained model - weights and updating the server model')
    updated_weights = receive_and_update_model(conn, model)
    print(updated_weights)
    client_weights_list.append(updated_weights)

all_weights_valid = val_weights(client_weights_list)
    
if client_counter == 3: #change to devices 3   
    print("""Aggregate weights from multiple models.""")
    # After collecting weights from all clients, aggregate them
    if all_weights_valid:
        aggregated_weights = aggregate_weights(client_weights_list)

        # Update the global model with the aggregated weights
        model.set_weights(aggregated_weights)
        print("Updated the global model with the aggregated weights")
        print(model)

        # send the global model back to each client
        for conn, address in client_conn:
            print("***************")
            print(f"global model weights sent to client {address}.")
            send_model(conn, model)
            conn.close()
        model.save('Globalmodel_Robost.h5')

server_socket.close()
print("Server program ending. All clients have been served.")





Server is listening on port 5300.
Building GM

Total number of elements in the GM weights: 3009
GM has been created and compiled successfully.
***************************************************************
Clients please connect to server


C:\Users\Lalitha\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


*************************************************
Established connection with client at ('127.0.0.1', 59345).
*******************
Initial model sent to client at ('127.0.0.1', 59345).
Ready for a new connection if any.
*******
*************************************************
Established connection with client at ('127.0.0.1', 59347).
*******************
Initial model sent to client at ('127.0.0.1', 59347).
Ready for a new connection if any.
*******
*************************************************
Established connection with client at ('127.0.0.1', 59349).
*******************
Initial model sent to client at ('127.0.0.1', 59349).
Ready for a new connection if any.
*******
Receiving client ('127.0.0.1', 59345) local trained model - weights and updating the server model
Server model has been updated with the client's weights.
[array([[ 6.38931036e-01, -2.34387353e-01,  3.17856491e-01,
         4.03195098e-02,  6.88581094e-02,  1.92862421e-01,
         2.92080522e-01,  6.00829087e-02, -8.

Server model has been updated with the client's weights.
[array([[ 1.50222719e+00, -2.25709110e-01, -2.56731212e-01,
        -5.97394645e-01,  2.69973606e-01,  1.89248055e-01,
        -5.65601707e-01, -1.36826694e+00, -1.11866400e-01,
        -3.93847078e-01,  2.44852126e-01, -1.81728303e-01,
         4.08191919e-01, -3.46863359e-01,  9.95582700e-01,
        -2.72306293e-01,  3.65529448e-01,  1.94907263e-01,
        -5.91909960e-02,  2.66417742e-01,  1.64640427e-01,
         6.19640589e-01,  1.67742357e-01, -2.10833892e-01,
        -4.79664236e-01,  7.41810501e-01, -5.95657170e-01,
        -1.28152966e+00, -2.69829422e-01,  4.94237214e-01,
        -1.95409685e-01, -5.57571590e-01,  8.23875546e-01,
         1.19140589e+00,  6.70032024e-01, -8.34289253e-01,
         3.68745685e-01, -4.77453351e-01,  1.50529787e-01,
         1.12310033e-02,  3.87244433e-01,  9.26329419e-02,
         4.35949750e-02, -2.26274416e-01,  1.68114558e-01,
        -7.58810520e-01, -2.71268070e-01,  2.33315423e-01